# Estudo de caso: Designação como um problema de fluxo de custo mínimo

Os solucionadores MIP e CP-SAT podem resolver uma classe de problemas maior do que o fluxo de custo mínimo, e, na maioria dos casos, MIP ou CP-SAT são as melhores escolhas.

No entanto, o solucionador de fluxo de custo mínimo pode muitas vezes retornar uma solução mais rápido do que o solucionador MIP ou CP-SAT. 

Você **DEVERÁ** usar o solucionador de fluxo de custo mínimo para resolver os problemas de designação abaixo. 

- Um problema de designação linear simples.
- Um problema de designação com equipes de trabalhadores.

## Como um problema de fluxo de custo mínimo representa um problema de designação?

Como o problema de fluxo de custo mínimo abaixo pode representar um problema de designação? Em primeiro lugar, como a capacidade de cada arco é 1, o suprimento de 4 na fonte força cada um dos quatro arcos que conduzem aos trabalhadores a ter um fluxo igual a 1.

Em seguida, a condição de fluxo de entrada ser igual ao fluxo de saída força o fluxo de saída de cada trabalhador a ser 1. Se possível, o solucionador direcionaria esse fluxo através do arco de custo mínimo que sai de cada trabalhador. 

No entanto, o solucionador não pode direcionar os fluxos de dois trabalhadores diferentes para uma única tarefa. Se isso acontecesse, haveria um fluxo combinado igual a 2 naquela tarefa, que não poderia ser enviado através do único arco com capacidade 1 da tarefa até o sumidouro. Isso significa que o solucionador só pode atribuir uma tarefa a um único trabalhador, conforme exigido pelo problema de designação.

Finalmente, a condição de fluxo de entrada igual a fluxo de saída força cada tarefa a ter um fluxo de saída igual a 1, de modo que cada tarefa é executada por algum trabalhador.

Para cada um das questões abaixo, implemente um programa em C++ usando o solucionador de fluxo de custo mínimo de forma a resolvê-los.

## Questão 1: problema de designação linear simples

No exemplo, há cinco trabalhadores (numerados de 1 a 4) e quatro tarefas (numeradas de 5 a 8).

<img src="./img/designacao-custo-minimo.png">

Os custos de atribuição de trabalhadores a tarefas são mostrados na tabela a seguir.

|   |5	|6	|7	|8  |
|---|---|---|---|---|
|1	|90	|76	|75	|70 |
|2	|35	|85	|55	|65 |
|3	|125|95	|90	|105 |
|4	|45	|110|95	|115|

In [1]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/graph/min_cost_flow.h"

using namespace operations_research;

{

    int numNodes = 10;
    int numArcs = 24;

    int startNodes[] = {0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 6, 7, 8};
    int endNodes[] = {1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 9, 9, 9, 9};
    int capacities[] = {1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1};
    int unitCosts[] = {0, 0, 0, 0, 90, 76, 75, 70, 35, 85, 55, 65, 125, 95, 90, 105, 45, 110, 95, 115, 0, 0, 0, 0};

    // Define an array of supplies at each node.
    int supplies[] = {4,0,0,0,0,0,0,0,0,-4};

    SimpleMinCostFlow minCostFlow;

    // Add each arc.
    for (int i = 0; i < numArcs; ++i) {
      int arc = minCostFlow.AddArcWithCapacityAndUnitCost(startNodes[i], endNodes[i],
                                           capacities[i], unitCosts[i]);
    }

     // Add node supplies.
    for (int i = 0; i < numNodes; ++i) {
      minCostFlow.SetNodeSupply(i, supplies[i]);
    }

    int solveStatus = minCostFlow.Solve();
    if (solveStatus == MinCostFlow::OPTIMAL) {
      auto optimalCost = minCostFlow.OptimalCost();
      std::cout << "Minimum cost: " << optimalCost << std::endl;

      for (int i = 0; i < numArcs; ++i) {
        auto cost = minCostFlow.Flow(i) * minCostFlow.UnitCost(i);
        if (cost > 0){
            std::cout << "TRABALHADOR:" << minCostFlow.Tail(i) << " -> " <<
                         "ATIVIDADE:" << minCostFlow.Head(i) << "  " <<
                         "CUSTO:"<<cost << std::endl;
        }
      }
    }
    else {
      std::cout << "Solving the min cost flow problem failed. Solver status: " << solveStatus;
    }
}

Minimum cost: 265
TRABALHADOR:1 -> ATIVIDADE:8  CUSTO:70
TRABALHADOR:2 -> ATIVIDADE:7  CUSTO:55
TRABALHADOR:3 -> ATIVIDADE:6  CUSTO:95
TRABALHADOR:4 -> ATIVIDADE:5  CUSTO:45


## Questão 2: problema de designação com equipes de trabalhadores

Nesse problema, seis trabalhadores são divididos em duas equipes. O problema é atribuir quatro tarefas aos trabalhadores para que a carga de trabalho seja igualmente equilibrada entre as equipes - ou seja, cada equipe executa duas das tarefas.

Os trabalhadores correspondem aos nós 1 - 6. A equipe A consiste nos trabalhadores 1, 3 e 5 e a equipe B consiste nos trabalhadores 2, 4 e 6. As tarefas são numeradas de 7 a 10.

Existem dois novos nós, 11 e 12, entre a origem e os trabalhadores. O nó 11 está conectado aos nós da equipe A e o nó 12 está conectado aos nós da equipe B, com arcos de capacidade 1. O gráfico a seguir mostra apenas os nós e arcos da origem aos trabalhadores.

<img src="./img/designacao-custo-minimo-equipes.png">

A chave para equilibrar a carga de trabalho é que a fonte 0 está conectada aos nós 11 e 12 por arcos de capacidade 2. Isso significa que os nós 11 e 12 (e, portanto, as equipes A e B) podem ter um fluxo máximo de 2. Como resultado, cada equipe pode realizar no máximo duas das tarefas.

Os custos de atribuição de trabalhadores a tarefas são mostrados na tabela a seguir.

|   |7	|8	|9	|10 |
|---|---|---|---|---|
|1	|90	|76	|75	|70 |
|2	|35	|85	|55	|65 |
|3	|125|95	|90	|105|
|4	|45	|110|95	|115|
|5  |60 |105| 80| 75|
|6  |45 | 65|110| 95|

**Observe que não aparece na figura o nó sumidouro 13 e nem os nós das tarefas (7-10) mas o grafo é semelhante a da questão 1, com arcos saindo de cada um dos seis trabalhadores para as tarefas e então para o sumidouro.**

In [2]:
using namespace operations_research;

{

    int numNodes = 14;
    int numArcs = 36;

    int startNodes[] = {0, 0,11, 11, 11, 12, 12, 12,1,1,1,1,2,2,2,2, 3, 3, 3, 3, 4, 4, 4, 4, 5,5,5,5, 6,6,6,6, 7, 8,9,10};
    int endNodes[] = {11,12,1,3,5,2,4,6, 7,8,9,10,7,8,9,10,7,8,9,10,7,8,9,10,7,8,9,10,7,8,9,10,13,13,13,13};
    int capacities[] = {2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,1,1,1,1,1,1,1,1,1,1,1,1};
    int unitCosts[] = {0,0,0,0,0,0,0,0,90,76,75,70,35,85,55,65,125,95,90,105,45,110,95,115,60,105,80,75,45,65,110,95,0,0,0,0};

    // Define an array of supplies at each node.
    int supplies[] = {4,0,0,0,0,0,0,0,0,0,0,0,0,-4};

    SimpleMinCostFlow minCostFlow;

    // Add each arc.
    for (int i = 0; i < numArcs; ++i) {
      int arc = minCostFlow.AddArcWithCapacityAndUnitCost(startNodes[i], endNodes[i],
                                           capacities[i], unitCosts[i]);
    }

     // Add node supplies.
    for (int i = 0; i < numNodes; ++i) {
      minCostFlow.SetNodeSupply(i, supplies[i]);
    }

    int solveStatus = minCostFlow.Solve();
    if (solveStatus == MinCostFlow::OPTIMAL) {
      auto optimalCost = minCostFlow.OptimalCost();
      std::cout << "Minimum cost: " << optimalCost << std::endl;

      for (int i = 0; i < numArcs; ++i) {
        if(minCostFlow.Flow(i)>0){
        auto cost = minCostFlow.Flow(i) * minCostFlow.UnitCost(i);
        if (cost > 0){
            if (minCostFlow.Tail(i) == 1 || minCostFlow.Tail(i) == 3 || minCostFlow.Tail(i) == 5){
                std::cout << "GRUPO 1 -> TRABALHADOR:" << minCostFlow.Tail(i) << " -> " <<
                              "ATIVIDADE:" << minCostFlow.Head(i) << "  " <<
                              "CUSTO:"<<cost << std::endl;
            }
            else{
                std::cout << "GRUPO 2 -> TRABALHADOR:" << minCostFlow.Tail(i) << " -> " <<
                             "ATIVIDADE:" << minCostFlow.Head(i) << "  " <<
                             "CUSTO:"<<cost << std::endl;
            }
        }
        }
      }
    }
    else {
      std::cout << "Solving the min cost flow problem failed. Solver status: " << solveStatus;
    }
}

Minimum cost: 250
GRUPO 1 -> TRABALHADOR:1 -> ATIVIDADE:9  CUSTO:75
GRUPO 2 -> TRABALHADOR:2 -> ATIVIDADE:7  CUSTO:35
GRUPO 1 -> TRABALHADOR:5 -> ATIVIDADE:10  CUSTO:75
GRUPO 2 -> TRABALHADOR:6 -> ATIVIDADE:8  CUSTO:65
